In [1]:
from main import main
from resultados import pipeline_graficos_resultados
import random
import gc
gc.collect()

0

## 1) Gerar Instância 

Variar:

* Quantidade de tarefas: estoque e docas
* Quantidade de máquinas
* Quantidade de caminhões

In [2]:
#random.seed(0)

####################### PARAMETROS VARIÁVEIS #######################

# numero de tarefas dos caminhoes de fornecedores
n_tarefas_docas = 3
# numero de tarefas dos estoques
n_tarefas_estoque = 2
# numero maquinas
n_maquinas = 2
# numero caminhoes 
n_caminhoes = 5
# define se todos os caminhoes ficarão 
todos_caminhoes_atrasados = False
todos_caminhoes_adiantados = False

####################### PARAMETROS FIXOS #######################

# define se os parametros serão deterministicos ou nao
deterministico = False
# velocidade mínima da empilhadeira rápida
vel_min_emp_rapida = 15
# velocidade máxima da empilhadeira rápida
vel_max_emp_rapida = 30
# velocidade mínima da empilhadeira lenta
vel_min_emp_lenta = 5
# velocidade máxima da empilhadeira lente
vel_max_emp_lenta = 20
# proporção de máquinas rápidas
proporcao_rapidas = 30 
# numero estoques
num_estoques = 6
# número de docas para caminhões de entrada e de saída
num_docas = 10
# comprimento da área de picking
comprimento_picking = 20
# define se todos os ponto de origem e destino no picking serão no mesmo lugar
mesmo_ponto_picking = True
# quantidade de operações por tarefa
n_operacoes_por_tarefa = 2
# tempo minimo de blocking
t_min_block = 5
# tempo máximo de blocking
t_max_block = 30
# tempo mínimo de setup
t_min_setup = 5
# tempo máximo de setup
t_max_setup = 50
# proporção de máquinas que podem atuar em diferentes áreas
proporcao_areas = {'1_area': 0.5, '2_areas': 0.3, '3_areas': 0.2}
# proporcao de maquinas para cada área. Deve totalizar 1
proporcao_maquinas = {'Docas entrada': 0.33, 'Estoque': 0.33, 'Picking': 0.33}
gc.collect()

0

In [3]:
area_indices, coordenadas_detalhadas, elegiblidade = main(num_estoques, 
                                            n_tarefas_estoque,
                                            n_tarefas_docas,
                                            num_docas, 
                                            comprimento_picking, 
                                            n_caminhoes, 
                                            mesmo_ponto_picking, 
                                            n_maquinas,  
                                            n_operacoes_por_tarefa, 
                                            proporcao_maquinas, 
                                            proporcao_areas,
                                            proporcao_rapidas,
                                            deterministico,
                                            vel_min_emp_rapida, 
                                            vel_max_emp_rapida, 
                                            vel_min_emp_lenta, 
                                            vel_max_emp_lenta, 
                                            t_min_block, 
                                            t_max_block, 
                                            t_min_setup, 
                                            t_max_setup, 
                                            todos_caminhoes_atrasados, 
                                            todos_caminhoes_adiantados,
                                            pasta='../data/instancias_sensitivity/',
                                            grid_spacing = 5)

In [6]:
# gerar_experimentos.py (Versão Final)

import pandas as pd
import numpy as np
from pyDOE2 import lhs

print("Iniciando a geração do Design of Experiments (DOE)...")

# --- 1. Definição dos Parâmetros do Experimento ---
# Limites superiores
MAX_TAREFAS = 50
MAX_MAQUINAS = 50
MAX_CAMINHOES = 50

# Limites inferiores (NOVA REGRA)
MIN_MAQUINAS = 5
MIN_CAMINHOES = 5
MIN_TAREFAS_ESTOQUE = 5
# n_tarefas_docas terá seu mínimo definido por n_caminhoes, que já é >= 5

# O número total mínimo de tarefas é a soma dos mínimos componentes
MIN_TAREFAS_TOTAL = MIN_CAMINHOES + MIN_TAREFAS_ESTOQUE # Ex: 5 + 5 = 10

# Número de amostras por condição
N_SAMPLES_PER_CONDITION = 20

# Semente aleatória para reprodutibilidade
np.random.seed(42)

# --- 2. Definição das Condições Booleanas ---
conditions = [
    {'todos_caminhoes_atrasados': False, 'todos_caminhoes_adiantados': False, 'condicao': 'Normal'},
    {'todos_caminhoes_atrasados': True,  'todos_caminhoes_adiantados': False, 'condicao': 'Atrasado'},
    {'todos_caminhoes_atrasados': False, 'todos_caminhoes_adiantados': True,  'condicao': 'Adiantado'}
]

# Lista para armazenar os DataFrames de cada condição
all_experiments_df_list = []

# --- 3. Geração dos Experimentos para cada Condição ---
for cond_info in conditions:
    print(f"Gerando amostras para a condição: '{cond_info['condicao']}'")
    
    # Gera um Latin Hypercube para 3 fatores (tarefas, máquinas, caminhões)
    lhd = lhs(3, samples=N_SAMPLES_PER_CONDITION, criterion='maximin')

    experiment_rows = []

    # --- 4. Escalonamento e Aplicação de TODAS as Restrições ---
    for i in range(N_SAMPLES_PER_CONDITION):
        # a. Gerar n_tarefas_total no novo intervalo [10, 100]
        n_tarefas_total = int(np.round(MIN_TAREFAS_TOTAL + lhd[i, 0] * (MAX_TAREFAS - MIN_TAREFAS_TOTAL)))
        
        # b. Gerar n_caminhoes no novo intervalo [5, max_allowed]
        max_allowed_caminhoes = min(MAX_CAMINHOES, n_tarefas_total - MIN_TAREFAS_ESTOQUE)
        # Garante que o intervalo de geração seja válido
        if max_allowed_caminhoes < MIN_CAMINHOES:
            max_allowed_caminhoes = MIN_CAMINHOES
        n_caminhoes = int(np.round(MIN_CAMINHOES + lhd[i, 2] * (max_allowed_caminhoes - MIN_CAMINHOES)))

        # c. Gerar n_maquinas no novo intervalo [5, max_allowed]
        max_allowed_maquinas = min(MAX_MAQUINAS, n_tarefas_total)
        if max_allowed_maquinas < MIN_MAQUINAS:
            max_allowed_maquinas = MIN_MAQUINAS
        n_maquinas = int(np.round(MIN_MAQUINAS + lhd[i, 1] * (max_allowed_maquinas - MIN_MAQUINAS)))

        # d. Gerar n_tarefas_docas e n_tarefas_estoque com as novas restrições
        limite_inferior_docas = n_caminhoes
        limite_superior_docas = n_tarefas_total - MIN_TAREFAS_ESTOQUE
        
        # Garante que limite_superior seja sempre >= limite_inferior
        if limite_superior_docas < limite_inferior_docas:
            limite_superior_docas = limite_inferior_docas

        n_tarefas_docas = np.random.randint(limite_inferior_docas, limite_superior_docas + 1)
        n_tarefas_estoque = n_tarefas_total - n_tarefas_docas
        
        # Adiciona a linha do experimento ao dicionário
        experiment_rows.append({
            'n_tarefas_docas': n_tarefas_docas,
            'n_tarefas_estoque': n_tarefas_estoque,
            'n_maquinas': n_maquinas,
            'n_caminhoes': n_caminhoes,
            'todos_caminhoes_atrasados': cond_info['todos_caminhoes_atrasados'],
            'todos_caminhoes_adiantados': cond_info['todos_caminhoes_adiantados'],
            'condicao': cond_info['condicao']
        })

    all_experiments_df_list.append(pd.DataFrame(experiment_rows))

# --- 5. Finalização e Salvamento ---
print("Combinando resultados e salvando o arquivo CSV...")

final_df = pd.concat(all_experiments_df_list, ignore_index=True)
final_df.insert(0, 'id_experimento', range(1, len(final_df) + 1))
final_df = final_df[[
    'id_experimento', 'condicao', 'n_tarefas_docas', 'n_tarefas_estoque', 
    'n_maquinas', 'n_caminhoes', 'todos_caminhoes_atrasados', 
    'todos_caminhoes_adiantados'
]]

output_filename = 'design_of_experiments_2.csv'
final_df.to_csv(output_filename, index=False)

print(f"\nArquivo '{output_filename}' gerado com sucesso!")
print(f"Total de {len(final_df)} experimentos criados.")

# --- 6. Verificação final das restrições ---
print("\nVerificando se todas as restrições foram atendidas:")
print(f"Mínimo de tarefas de doca >= 5: {(final_df['n_tarefas_docas'] >= 5).all()}")
print(f"Mínimo de tarefas de estoque >= 5: {(final_df['n_tarefas_estoque'] >= 5).all()}")
print(f"Mínimo de máquinas >= 5: {(final_df['n_maquinas'] >= 5).all()}")
print(f"Mínimo de caminhões >= 5: {(final_df['n_caminhoes'] >= 5).all()}")
print(f"Tarefas de doca >= Caminhões: {(final_df['n_tarefas_docas'] >= final_df['n_caminhoes']).all()}")

Iniciando a geração do Design of Experiments (DOE)...
Gerando amostras para a condição: 'Normal'
Gerando amostras para a condição: 'Atrasado'
Gerando amostras para a condição: 'Adiantado'
Combinando resultados e salvando o arquivo CSV...

Arquivo 'design_of_experiments_2.csv' gerado com sucesso!
Total de 60 experimentos criados.

Verificando se todas as restrições foram atendidas:
Mínimo de tarefas de doca >= 5: True
Mínimo de tarefas de estoque >= 5: True
Mínimo de máquinas >= 5: True
Mínimo de caminhões >= 5: True
Tarefas de doca >= Caminhões: True


In [4]:
# executar_experimentos.py

import pandas as pd
import gc

# =======================================================================
# ADICIONE A IMPORTAÇÃO DA SUA FUNÇÃO 'main' AQUI
# Exemplo:
# from seu_arquivo_de_simulacao import main
# =======================================================================


# --- PARÂMETROS FIXOS ---
# Parâmetros que não mudam entre os experimentos.
PARAMETROS_FIXOS = {
    'num_estoques': 6,
    'num_docas': 10,
    'comprimento_picking': 20,
    'mesmo_ponto_picking': True,
    'n_operacoes_por_tarefa': 2,
    'proporcao_maquinas': {'Docas entrada': 0.33, 'Estoque': 0.33, 'Picking': 0.33},
    'proporcao_areas': {'1_area': 0.5, '2_areas': 0.3, '3_areas': 0.2},
    'proporcao_rapidas': 30,
    'deterministico': False,
    'vel_min_emp_rapida': 15,
    'vel_max_emp_rapida': 30,
    'vel_min_emp_lenta': 5,
    'vel_max_emp_lenta': 20,
    't_min_block': 5,
    't_max_block': 30,
    't_min_setup': 5,
    't_max_setup': 50,
    'pasta': '../data/instancias_sensitivity_5/',
    'grid_spacing': 5
}

# --- CARREGAMENTO DO PLANO DE EXPERIMENTOS ---
try:
    df_experimentos = pd.read_csv('design_of_experiments_iterativo_caminhoes.csv')
except FileNotFoundError:
    print("ERRO: Arquivo 'design_of_experiments_iterativo_caminhoes.csv' não encontrado.")
    print("Por favor, execute o script 'gerar_experimentos.py' primeiro.")
    exit()

print(f"Arquivo de experimentos carregado. Iniciando a execução de {len(df_experimentos)} instâncias...")

# --- LOOP PRINCIPAL DE EXECUÇÃO ---
for index, experimento in df_experimentos.iterrows():
    
    id_atual = experimento['id_experimento']
    print(f"\n--- Executando Experimento ID: {id_atual} (Condição: {experimento['condicao']}) ---")

    try:
            
        # Chamada da função 'main' com os parâmetros fixos e os variáveis (lidos do CSV)
        area_indices, coordenadas_detalhadas, elegibilidade = main(
            # Parâmetros Fixos:
            num_estoques=PARAMETROS_FIXOS['num_estoques'],
            num_docas=PARAMETROS_FIXOS['num_docas'],
            picking_width_units=PARAMETROS_FIXOS['comprimento_picking'],
            mesmo_ponto_picking=PARAMETROS_FIXOS['mesmo_ponto_picking'],
            n_operacoes_por_tarefa=PARAMETROS_FIXOS['n_operacoes_por_tarefa'],
            proporcao_maquinas=PARAMETROS_FIXOS['proporcao_maquinas'],
            proporcao_areas=PARAMETROS_FIXOS['proporcao_areas'],
            proporcao_rapidas=PARAMETROS_FIXOS['proporcao_rapidas'],
            deterministico=PARAMETROS_FIXOS['deterministico'],
            vel_min_emp_rapida=PARAMETROS_FIXOS['vel_min_emp_rapida'],
            vel_max_emp_rapida=PARAMETROS_FIXOS['vel_max_emp_rapida'],
            vel_min_emp_lenta=PARAMETROS_FIXOS['vel_min_emp_lenta'],
            vel_max_emp_lenta=PARAMETROS_FIXOS['vel_max_emp_lenta'],
            t_min_block=PARAMETROS_FIXOS['t_min_block'],
            t_max_block=PARAMETROS_FIXOS['t_max_block'],
            t_min_setup=PARAMETROS_FIXOS['t_min_setup'],
            t_max_setup=PARAMETROS_FIXOS['t_max_setup'],
            pasta=PARAMETROS_FIXOS['pasta'],
            grid_spacing=PARAMETROS_FIXOS['grid_spacing'],
            
            # Parâmetros Variáveis (da linha atual do CSV):
            n_tarefas_estoque=int(experimento['n_tarefas_estoque']),
            n_tarefas_docas=int(experimento['n_tarefas_docas']),
            n_caminhoes=int(experimento['n_caminhoes']),
            n_maquinas=int(experimento['n_maquinas']),
            todos_caminhoes_atrasados=bool(experimento['todos_caminhoes_atrasados']),
            todos_caminhoes_adiantados=bool(experimento['todos_caminhoes_adiantados'])
        )
        
        print(f"Experimento {id_atual} concluído com sucesso.")
        
    except NameError:
        print("\nERRO CRÍTICO: A função 'main' não foi encontrada.")
        print("Verifique se você adicionou a linha de importação no topo do script (ex: 'from seu_arquivo import main').")
        break # Para o loop imediatamente
    except Exception as e:
        print(f"!!!!!!!!!!!!!!! ERRO AO EXECUTAR O EXPERIMENTO {id_atual} !!!!!!!!!!!!!!!")
        print(f"Tipo de erro: {type(e).__name__}, Mensagem: {e}")
        continue # Continua para o próximo experimento
        
    # Coleta de lixo para liberar memória.
    gc.collect()

print("\n==================================================")
print("TODOS OS EXPERIMENTOS FORAM PROCESSADOS.")
print("==================================================")

Arquivo de experimentos carregado. Iniciando a execução de 75 instâncias...

--- Executando Experimento ID: 1 (Condição: Normal) ---
Experimento 1 concluído com sucesso.

--- Executando Experimento ID: 2 (Condição: Normal) ---
Experimento 2 concluído com sucesso.

--- Executando Experimento ID: 3 (Condição: Normal) ---
Experimento 3 concluído com sucesso.

--- Executando Experimento ID: 4 (Condição: Normal) ---
Experimento 4 concluído com sucesso.

--- Executando Experimento ID: 5 (Condição: Normal) ---
Experimento 5 concluído com sucesso.

--- Executando Experimento ID: 6 (Condição: Normal) ---
Experimento 6 concluído com sucesso.

--- Executando Experimento ID: 7 (Condição: Normal) ---
Experimento 7 concluído com sucesso.

--- Executando Experimento ID: 8 (Condição: Normal) ---
Experimento 8 concluído com sucesso.

--- Executando Experimento ID: 9 (Condição: Normal) ---
Experimento 9 concluído com sucesso.

--- Executando Experimento ID: 10 (Condição: Normal) ---
Experimento 10 concl

## 2) Gerar Resultados

In [3]:
# numero de tarefas dos caminhoes de fornecedores
n_tarefas_docas = 20
# numero de tarefas dos estoques
n_tarefas_estoque = 20
# numero maquinas
n_maquinas = 15
# numero caminhoes 
n_caminhoes = 10
# define se todos os caminhoes ficarão 
todos_caminhoes_atrasados = False
todos_caminhoes_adiantados = False

if todos_caminhoes_atrasados:
    file_path = f'../data/resultados_completos/result_{n_tarefas_docas}_{n_tarefas_estoque}_{n_maquinas}_{n_caminhoes}_at.txt'

elif todos_caminhoes_adiantados:
    file_path = f'../data/resultados_completos/result_{n_tarefas_docas}_{n_tarefas_estoque}_{n_maquinas}_{n_caminhoes}_ad.txt'
else:
    file_path = f'../data/resultados_completos/result_{n_tarefas_docas}_{n_tarefas_estoque}_{n_maquinas}_{n_caminhoes}.txt'

area_indices = None
coordenadas_detalhadas = None
pipeline_graficos_resultados(file_path, area_indices, coordenadas_detalhadas)
gc.collect()

Erro ao calcular as métricas: 'n_caminhoes'
Erro ao gerar gráfico de Gantt das empilhadeiras: 'n_caminhoes'
Erro ao gerar gráfico de Gantt dos caminhões: 'n_caminhoes'
Erro ao gerar gráfico de Gantt por tarefas: 'n_caminhoes'
Erro ao gerar o mapa de calor: 'NoneType' object has no attribute 'values'


0

In [4]:
from collections import defaultdict
import numpy as np
import re

def parse_mac(lines):
    for line in lines:
        if line.startswith('MAC ='):
            return int(line.split('=')[1].strip())
    return None

def parse_machines(lines):
    for line in lines:
        if line.startswith('n_machines ='):
            return int(line.split('=')[1].strip())
    return None

def parse_caminhoes(lines):
    for line in lines:
        if line.startswith('n_caminhoes ='):
            return int(line.split('=')[1].strip())
    return None

def parse_alpha_caminhao(lines):
    """
    Função para parsing de dados de alpha para caminhões no formato [*,num,*] 
    que lidam com quebras de páginas.
    """
    alpha = defaultdict(dict)  # {truck_number: numpy_array}
    collecting = False  # Flag para indicar quando estamos coletando dados
    truck_number = None  # Número do caminhão atual
    current_truck_data = []  # Dados temporários para um caminhão
    additional_columns = []  # Colunas adicionais para concatenar

    i = 0
    while i < len(lines):
        line = lines[i].strip()

        # Início da primeira ou subsequente seção alpha para um caminhão específico
        if re.match(r'alpha \[\*,\d+,\*\]', line) or re.match(r'\[\*,\d+,\*\]', line):
            # Finaliza o bloco anterior, se houver, e inicia um novo
            if collecting and current_truck_data:
                if additional_columns:
                    # Adiciona as colunas adicionais ao final do bloco atual
                    current_truck_data = [row + add_row for row, add_row in zip(current_truck_data, additional_columns)]
                    additional_columns = []
                
                alpha[truck_number] = np.array(current_truck_data)
                current_truck_data = []  # Reinicializa os dados para o próximo caminhão
            
            # Extrair o número do caminhão
            truck_number = int(re.search(r'\d+', line).group())
            collecting = True  # Começar a coletar dados
            i += 2  # Pular a linha de cabeçalho e ir para a primeira linha de dados
            continue

        # Coletar os dados enquanto não encontrar o delimitador ';'
        if collecting:
            if line.startswith(';'):  # Fim da seção alpha
                if additional_columns:
                    # Adiciona colunas adicionais ao bloco atual antes de finalizar
                    current_truck_data = [row + add_row for row, add_row in zip(current_truck_data, additional_columns)]
                    additional_columns = []
                
                alpha[truck_number] = np.array(current_truck_data)  # Armazena os dados finais do caminhão
                collecting = False
                current_truck_data = []  # Limpa os dados temporários
            else:
                parts = line.split()
                # Verifica se a linha tem dados válidos para o alpha
                if len(parts) > 1 and all(x.isdigit() for x in parts):
                    row_data = [int(x) for x in parts[1:]]  # Coleta a linha de dados e ignora a primeira coluna
                    if len(current_truck_data) < len(row_data):
                        current_truck_data.append(row_data)
                    else:
                        additional_columns.append(row_data)
            i += 1
        else:
            i += 1

    # Finaliza o último bloco, caso ele termine sem um delimitador ';'
    if collecting and current_truck_data:
        if additional_columns:
            current_truck_data = [row + add_row for row, add_row in zip(current_truck_data, additional_columns)]
        alpha[truck_number] = np.array(current_truck_data)

    return alpha

def parse_alpha_empilhadeira(lines):
    """
    Função para parsing de dados de alpha para empilhadeiras no formato [*,*,num] 
    que lida com quebras de páginas.
    """
    alpha = defaultdict(dict)  # {forklift_number: numpy_array}
    collecting = False  # Flag para indicar quando estamos coletando dados
    forklift_number = None  # Número da empilhadeira atual
    current_forklift_data = []  # Dados temporários para uma empilhadeira
    additional_columns = []  # Colunas adicionais para concatenar

    i = 0
    while i < len(lines):
        line = lines[i].strip()

        # Início da primeira ou subsequente seção alpha para uma empilhadeira específica
        if re.match(r'alpha \[\*,\*,\d+\]', line) or re.match(r'\[\*,\*,\d+\]', line):
            # Finaliza o bloco anterior, se houver, e inicia um novo
            if collecting and current_forklift_data:
                if additional_columns:
                    # Adiciona colunas adicionais ao final do bloco atual
                    current_forklift_data = [row + add_row for row, add_row in zip(current_forklift_data, additional_columns)]
                    additional_columns = []
                
                alpha[forklift_number] = np.array(current_forklift_data).T  # Transpõe para colunas de caminhões
                current_forklift_data = []  # Reinicializa os dados para a próxima empilhadeira
            
            # Extrair o número da empilhadeira
            forklift_number = int(re.search(r'\d+', line).group())
            collecting = True  # Começar a coletar dados
            i += 2  # Pular a linha de cabeçalho e ir para a primeira linha de dados
            continue

        # Coletar os dados enquanto não encontrar o delimitador ';'
        if collecting:
            if line.startswith(';'):  # Fim da seção alpha
                if additional_columns:
                    # Adiciona colunas adicionais ao bloco atual antes de finalizar
                    current_forklift_data = [row + add_row for row, add_row in zip(current_forklift_data, additional_columns)]
                    additional_columns = []
                
                alpha[forklift_number] = np.array(current_forklift_data).T  # Transpõe para colunas de caminhões
                collecting = False
                current_forklift_data = []  # Limpa os dados temporários
            else:
                parts = line.split()
                # Verifica se a linha tem dados válidos para o alpha
                if len(parts) > 1 and all(x.isdigit() for x in parts):
                    row_data = [int(x) for x in parts[1:]]  # Coletar a linha de dados e ignorar a primeira coluna
                    if len(current_forklift_data) < len(row_data):
                        current_forklift_data.append(row_data)
                    else:
                        additional_columns.append(row_data)
            i += 1
        else:
            i += 1

    # Finaliza o último bloco, caso ele termine sem um delimitador ';'
    if collecting and current_forklift_data:
        if additional_columns:
            current_forklift_data = [row + add_row for row, add_row in zip(current_forklift_data, additional_columns)]
        alpha[forklift_number] = np.array(current_forklift_data).T

    return alpha

def parse_t_and_A(lines):
    t = {}
    A = {}
    i = 0

    while i < len(lines):
        line = lines[i].strip()
        if line.startswith(':') and 't' in line and 'A' in line:
            i += 1
            while i < len(lines):
                line = lines[i].strip()
                if line == ';' or line == '':
                    break
                parts = line.split()
                if len(parts) >= 3:
                    op_num = int(parts[0])
                    t_value = float(parts[1])
                    A_value = parts[2]
                    t[op_num] = t_value
                    if A_value != '.':
                        A[op_num] = float(A_value)
                i += 1
        i += 1
    return t, A

def parse_processing_time(lines):
    p = []
    i = 0

    while i < len(lines):
        line = lines[i].strip()
        if line.startswith('p [*,*]'):
            i += 1
            while i < len(lines):
                line = lines[i].strip()
                if line == ';' or line == '':  # Encerramento da seção de p
                    break
                if line.startswith(':') or line.startswith(':='):
                    # Ignorar linhas de cabeçalho que começam com ':' ou ':='
                    i += 1
                    continue
                
                parts = line.split()
                
                if len(parts) > 1:  # Certifica-se de que há pelo menos 1 valor de operação
                    try:
                        op_num = int(parts[0])  # Primeira coluna é o número da operação
                    except ValueError:
                        i += 1
                        continue  # Pula a linha se a conversão falhar

                    for j, time_value in enumerate(parts[1:], start=1):  # Itera sobre os tempos
                        if time_value != '.':  # Ignora valores faltantes
                            p.append((op_num, j, float(time_value)))
                i += 1
        else:
            i += 1
    return p

def parse_d(lines):
    d = {}
    i = 0

    while i < len(lines):
        line = lines[i].strip()
        if line.startswith('d [*] :='):
            i += 1  # Move to the next line (data lines)
            while i < len(lines):
                line = lines[i].strip()
                if line.startswith(';') or line == '':
                    break
                parts = line.split()
                if len(parts) == 2:
                    truck_num = int(parts[0])
                    departure_time = float(parts[1])
                    d[truck_num] = departure_time
                i += 1
        i += 1
    return d

def parse_lines(filename):
    with open(filename, 'r') as file:
        lines = [line.rstrip('\n') for line in file]
    
    return lines

def parse_log_file(filename):
    with open(filename, 'r') as file:
        lines = [line.rstrip('\n') for line in file]
    
    # Chamar as funções independentes para ler cada parâmetro
    mac = parse_mac(lines)
    alpha = parse_alpha_caminhao(lines)
    if not alpha:  # ou if len(alpha) == 0:
        alpha = parse_alpha_empilhadeira(lines)
    t, A = parse_t_and_A(lines)
    processing_times = parse_processing_time(lines)
    n_maquinas = parse_machines(lines)
    n_caminhoes = parse_caminhoes(lines)
    departure_times = parse_d(lines)  # Chamar a nova função para o parâmetro 'd'
    
    return {
        'MAC': mac,
        'alpha': alpha,
        't': t,
        'A': A,
        'p': processing_times,
        'd': departure_times,
        'n_maquinas': n_maquinas,
        'n_caminhoes': n_caminhoes
    }

result = parse_log_file(file_path)

In [5]:
result['alpha']

defaultdict(dict,
            {1: array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0,

In [70]:
import re
import numpy as np
from collections import defaultdict

def parse_alpha_caminhao(lines):
    """
    Função para parsing de dados de alpha para caminhões no formato [*,num,*] 
    que lidam com quebras de páginas.
    """
    alpha = defaultdict(dict)  # {truck_number: numpy_array}
    collecting = False  # Flag para indicar quando estamos coletando dados
    truck_number = None  # Número do caminhão atual
    current_truck_data = []  # Dados temporários para um caminhão
    additional_columns = []  # Colunas adicionais para concatenar

    i = 0
    while i < len(lines):
        line = lines[i].strip()

        # Início da primeira ou subsequente seção alpha para um caminhão específico
        if re.match(r'alpha \[\*,\d+,\*\]', line) or re.match(r'\[\*,\d+,\*\]', line):
            # Finaliza o bloco anterior, se houver, e inicia um novo
            if collecting and current_truck_data:
                if additional_columns:
                    # Adiciona as colunas adicionais ao final do bloco atual
                    current_truck_data = [row + add_row for row, add_row in zip(current_truck_data, additional_columns)]
                    additional_columns = []
                
                alpha[truck_number] = np.array(current_truck_data)
                current_truck_data = []  # Reinicializa os dados para o próximo caminhão
            
            # Extrair o número do caminhão
            truck_number = int(re.search(r'\d+', line).group())
            collecting = True  # Começar a coletar dados
            i += 2  # Pular a linha de cabeçalho e ir para a primeira linha de dados
            continue

        # Coletar os dados enquanto não encontrar o delimitador ';'
        if collecting:
            if line.startswith(';'):  # Fim da seção alpha
                if additional_columns:
                    # Adiciona colunas adicionais ao bloco atual antes de finalizar
                    current_truck_data = [row + add_row for row, add_row in zip(current_truck_data, additional_columns)]
                    additional_columns = []
                
                alpha[truck_number] = np.array(current_truck_data)  # Armazena os dados finais do caminhão
                collecting = False
                current_truck_data = []  # Limpa os dados temporários
            else:
                parts = line.split()
                # Verifica se a linha tem dados válidos para o alpha
                if len(parts) > 1 and all(x.isdigit() for x in parts):
                    row_data = [int(x) for x in parts[1:]]  # Coleta a linha de dados e ignora a primeira coluna
                    if len(current_truck_data) < len(row_data):
                        current_truck_data.append(row_data)
                    else:
                        additional_columns.append(row_data)
            i += 1
        else:
            i += 1

    # Finaliza o último bloco, caso ele termine sem um delimitador ';'
    if collecting and current_truck_data:
        if additional_columns:
            current_truck_data = [row + add_row for row, add_row in zip(current_truck_data, additional_columns)]
        alpha[truck_number] = np.array(current_truck_data)

    return alpha

def parse_alpha_empilhadeira(lines):
    """
    Função para parsing de dados de alpha para empilhadeiras no formato [*,*,num] 
    que lida com quebras de páginas.
    """
    alpha = defaultdict(dict)  # {forklift_number: numpy_array}
    collecting = False  # Flag para indicar quando estamos coletando dados
    forklift_number = None  # Número da empilhadeira atual
    current_forklift_data = []  # Dados temporários para uma empilhadeira
    additional_columns = []  # Colunas adicionais para concatenar

    i = 0
    while i < len(lines):
        line = lines[i].strip()

        # Início da primeira ou subsequente seção alpha para uma empilhadeira específica
        if re.match(r'alpha \[\*,\*,\d+\]', line) or re.match(r'\[\*,\*,\d+\]', line):
            # Finaliza o bloco anterior, se houver, e inicia um novo
            if collecting and current_forklift_data:
                if additional_columns:
                    # Adiciona colunas adicionais ao final do bloco atual
                    current_forklift_data = [row + add_row for row, add_row in zip(current_forklift_data, additional_columns)]
                    additional_columns = []
                
                alpha[forklift_number] = np.array(current_forklift_data).T  # Transpõe para colunas de caminhões
                current_forklift_data = []  # Reinicializa os dados para a próxima empilhadeira
            
            # Extrair o número da empilhadeira
            forklift_number = int(re.search(r'\d+', line).group())
            collecting = True  # Começar a coletar dados
            i += 2  # Pular a linha de cabeçalho e ir para a primeira linha de dados
            continue

        # Coletar os dados enquanto não encontrar o delimitador ';'
        if collecting:
            if line.startswith(';'):  # Fim da seção alpha
                if additional_columns:
                    # Adiciona colunas adicionais ao bloco atual antes de finalizar
                    current_forklift_data = [row + add_row for row, add_row in zip(current_forklift_data, additional_columns)]
                    additional_columns = []
                
                alpha[forklift_number] = np.array(current_forklift_data).T  # Transpõe para colunas de caminhões
                collecting = False
                current_forklift_data = []  # Limpa os dados temporários
            else:
                parts = line.split()
                # Verifica se a linha tem dados válidos para o alpha
                if len(parts) > 1 and all(x.isdigit() for x in parts):
                    row_data = [int(x) for x in parts[1:]]  # Coletar a linha de dados e ignorar a primeira coluna
                    if len(current_forklift_data) < len(row_data):
                        current_forklift_data.append(row_data)
                    else:
                        additional_columns.append(row_data)
            i += 1
        else:
            i += 1

    # Finaliza o último bloco, caso ele termine sem um delimitador ';'
    if collecting and current_forklift_data:
        if additional_columns:
            current_forklift_data = [row + add_row for row, add_row in zip(current_forklift_data, additional_columns)]
        alpha[forklift_number] = np.array(current_forklift_data).T

    return alpha

In [74]:
# numero de tarefas dos caminhoes de fornecedores
n_tarefas_docas = 30
# numero de tarefas dos estoques
n_tarefas_estoque = 30
# numero maquinas
n_maquinas = 30
# numero caminhoes 
n_caminhoes = 15
# define se todos os caminhoes ficarão 
todos_caminhoes_atrasados = False
todos_caminhoes_adiantados = False

if todos_caminhoes_atrasados:
    file_path = f'../data/resultados/result_{n_tarefas_docas}_{n_tarefas_estoque}_{n_maquinas}_{n_caminhoes}_at.txt'

elif todos_caminhoes_adiantados:
    file_path = f'../data/resultados/result_{n_tarefas_docas}_{n_tarefas_estoque}_{n_maquinas}_{n_caminhoes}_ad.txt'
else:
    file_path = f'../data/resultados/result_{n_tarefas_docas}_{n_tarefas_estoque}_{n_maquinas}_{n_caminhoes}.txt'

with open(file_path, 'r') as file:
    lines = [line.rstrip('\n') for line in file]


In [75]:
parse_alpha_caminhao(lines)

defaultdict(dict, {})

In [76]:
parse_alpha_empilhadeira(lines)

defaultdict(dict,
            {1: array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0